In [1]:
import random
import math
import heapq


In [2]:

class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y
class Cell:
    def __init__(self, position, parent=None, status='free', g=0, h=0, f=0):
        self.position = position   
        self.parent = parent       
        self.status = status       # {'free', 'start', 'dirty', 'clean'}

        # các chi phí
        self.g = g   
        self.h = h  
        self.f = f   
        # cphí làm sạch ( = 1, tăng thêm sau mỗi hành động)
        self.cleaning_cost = 1
    def move_cost(self):
        return 1
    def update_f(self):
        self.f = self.g + self.h

    def __eq__(self, other):
        return (self.position.x == other.position.x and self.position.y == other.position.y)

    def __repr__(self):
        return (f"Cell({self.position.x}, {self.position.y}, "
                f"status={self.status}, g={self.g}, h={self.h}, f={self.f}, "
                f"clean_cost={self.cleaning_cost})")

    def calculate_h(self, goal):
        self.h = math.sqrt((self.position.x - goal.position.x)**2 + (self.position.y - goal.position.y)**2)
        self.update_f()
    
    def increase_cleaning_cost(self):
        if self.status == 'dirty':
            self.cleaning_cost += 1

    def apply_cleaning_cost(self):
        if self.status == 'dirty':
            self.g += self.cleaning_cost
            self.update_f()
            self.status = 'clean'
     

In [ ]:
class RobotCleaner:
    def __init__(self, rows, cols, num_dirty, num_obstacles):
        self.rows = rows
        self.cols = cols
        self.num_dirty = num_dirty
        self.num_obstacles = num_obstacles  # số lượng chướng ngại vật do người dùng nhập

        self.grid = []
        self.robot_pos = None
        self.dirty_cells = []

        self.total_move_cost = 0
        self.total_cleaning_cost = 0  # tổng chi phí tăng dần của các ô bẩn
        self.initialize_grid()

    def initialize_grid(self):
        self.grid = [[Cell(Position(x, y)) for y in range(self.cols)] for x in range(self.rows)]

     
        rx, ry = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
        self.robot_pos = Position(rx, ry)
        self.grid[rx][ry].status = 'start'

        # Đặt chướng ngại vật
        count = 0
        while count < self.num_obstacles:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'obstacle'
                count += 1

        # Đặt ô bẩn ngẫu nhiên
        count = 0
        while count < self.num_dirty:
            x, y = random.randint(0, self.rows - 1), random.randint(0, self.cols - 1)
            if self.grid[x][y].status == 'free' and (x != rx or y != ry):
                self.grid[x][y].status = 'dirty'
                self.dirty_cells.append(self.grid[x][y])
                count += 1

    def heuristic(self, pos1, pos2):
        # Chebyshev (8 hướng)
        return max(abs(pos1.x - pos2.x), abs(pos1.y - pos2.y))

    def get_neighbors(self, cell):
        directions = [
            (-1, -1), (-1, 0), (-1, 1),
            (0, -1),           (0, 1),
            (1, -1),  (1, 0),  (1, 1)
        ]
        neighbors = []
        for dx, dy in directions:
            nx, ny = cell.position.x + dx, cell.position.y + dy
            if 0 <= nx < self.rows and 0 <= ny < self.cols:
                neighbor = self.grid[nx][ny]
                if neighbor.status != 'obstacle':  # tránh vật cản
                    neighbors.append(neighbor)
        return neighbors

    def reset_cells(self):
        for x in range(self.rows):
            for y in range(self.cols):
                c = self.grid[x][y]
                c.g = c.h = c.f = 0
                c.parent = None

    def find_path(self, start, goal):
        self.reset_cells()
        start_cell = self.grid[start.x][start.y]
        goal_cell = self.grid[goal.x][goal.y]

        open_list = []
        closed = set()
        counter = 0

        start_cell.g = 0
        start_cell.h = self.heuristic(start, goal)
        start_cell.f = start_cell.g + start_cell.h
        heapq.heappush(open_list, (start_cell.f, counter, start_cell))

        while open_list:
            f, _, current = heapq.heappop(open_list)

            if current.position.x == goal.x and current.position.y == goal.y:
                path = []
                while current:
                    path.append(current.position)
                    current = current.parent
                return path[::-1]

            closed.add((current.position.x, current.position.y))

            for neighbor in self.get_neighbors(current):
                if (neighbor.position.x, neighbor.position.y) in closed:
                    continue

                tentative_g = current.g + neighbor.move_cost()
                if neighbor.parent is None or tentative_g < neighbor.g:
                    neighbor.parent = current
                    neighbor.g = tentative_g
                    neighbor.h = self.heuristic(neighbor.position, goal)
                    neighbor.update_f()
                    counter += 1
                    heapq.heappush(open_list, (neighbor.f, counter, neighbor))
        return None

    def print_map(self, current_robot_pos=None):
        rp = current_robot_pos if current_robot_pos else self.robot_pos
        for x in range(self.rows):
            row = []
            for y in range(self.cols):
                cell = self.grid[x][y]
                if rp.x == x and rp.y == y:
                    row.append("R")
                elif cell.status == "dirty":
                    row.append("D")
                elif cell.status == "clean":
                    row.append("C")
                elif cell.status == "start":
                    row.append("S")
                elif cell.status == "obstacle":
                    row.append("O")
                else:
                    row.append(".")
            print(" ".join(row))
        print()

    def clean_all(self):
        current_pos = self.robot_pos
        print("Bản đồ ban đầu:")
        self.print_map(current_pos)
        print('-----------------------------------------')

        while self.dirty_cells:
            nearest = min(self.dirty_cells, key=lambda d: self.heuristic(current_pos, d.position))
            path = self.find_path(current_pos, nearest.position)
            if not path:
                print(f" Không tìm được đường đến ô ({nearest.position.x}, {nearest.position.y}) - bị chặn bởi chướng ngại vật")
                self.dirty_cells.remove(nearest)
                continue

            for step_pos in path[1:]:
                self.total_move_cost += 1
                for cell in self.dirty_cells:
                    cell.increase_cleaning_cost()
                current_pos = step_pos
                self.print_map(current_pos)

            nearest.apply_cleaning_cost()
            self.total_cleaning_cost += nearest.cleaning_cost
            self.dirty_cells.remove(nearest)
            current_pos = nearest.position
            self.robot_pos = current_pos

            for cell in self.dirty_cells:
                cell.increase_cleaning_cost()

            print(f"Hút bụi tại ô ({nearest.position.x}, {nearest.position.y})")
            self.print_map(current_pos)
            print(f"Chi phí di chuyển hiện tại: {self.total_move_cost}")
            print(f"Tổng chi phí làm sạch cộng dồn: {self.total_cleaning_cost}")
            print('-----------------------------------------')

        print("Hoàn thành làm sạch tất cả ô!")
        print(f"Chi phí di chuyển: {self.total_move_cost}")
        print(f"Chi phí làm sạch: {self.total_cleaning_cost}")
        print(f"Tổng chi phí cuối cùng: {self.total_move_cost + self.total_cleaning_cost}")

In [4]:
if __name__ == "__main__":
    m = int(input("Nhập số hàng: "))
    n = int(input("Nhập số cột: "))
    num_dirty = int(input("Nhập số ô bẩn: "))
    num_obstacles = int(input("Nhập số chướng ngại vật: "))

    robot = RobotCleaner(m, n, num_dirty, num_obstacles)
    robot.clean_all()

Bản đồ ban đầu:
O . . . .
. . D . .
. . . D .
R . O . .
. D . . .

-----------------------------------------
O . . . .
. . D . .
. . . D .
S . O . .
. R . . .

Hút bụi tại ô (4, 1)
O . . . .
. . D . .
. . . D .
S . O . .
. R . . .

Chi phí di chuyển hiện tại: 1
Tổng chi phí làm sạch cộng dồn: 2
-----------------------------------------
O . . . .
. . D . .
. . . D .
S R O . .
. C . . .

O . . . .
. . D . .
. . R D .
S . O . .
. C . . .

O . . . .
. . D . .
. . . R .
S . O . .
. C . . .

Hút bụi tại ô (2, 3)
O . . . .
. . D . .
. . . R .
S . O . .
. C . . .

Chi phí di chuyển hiện tại: 4
Tổng chi phí làm sạch cộng dồn: 8
-----------------------------------------
O . . . .
. . R . .
. . . C .
S . O . .
. C . . .

Hút bụi tại ô (1, 2)
O . . . .
. . R . .
. . . C .
S . O . .
. C . . .

Chi phí di chuyển hiện tại: 5
Tổng chi phí làm sạch cộng dồn: 16
-----------------------------------------
Hoàn thành làm sạch tất cả ô!
Chi phí di chuyển: 5
Chi phí làm sạch: 16
 Tổng chi phí cuối cùng: 21
